In [ ]:
###--- Standard Imports ---###
import os
import pandas as pd
import numpy as np
from datetime import datetime as dt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

PATH = os.getcwd() #Pulls your current working directory
FILE = 'Customers_Cleaned.csv'
FULL_PATH = os.path.join(PATH, FILE) #Joins directory and file as one string

###--- Global Variables ---###
#Plan details: cost-per-month(USD), duration(years)
subscriber_plan = {
    'Basic': [10, 1],
    'Gold': [15, 1],
    'Platinum': [20, 1],
    'Obsidian': [30, 2],
    'Infinity': [30, 5]
}

###--- Global Functions ---###
#Convert string-value dates to ordinal dates (integers) --> takes a 1D array
def date_to_ordinal(data_series):
    from datetime import datetime as dt

    ord_out = [dt.toordinal(dt.strptime(i, "%Y-%m-%d")) for i in data_series]
    return ord_out #Returns 1D ordinal array

#Find the difference beween two ordinal dates
def date_diff(dataframe, start_col=False, end_col=False, ytd=False, date_math=lambda ed, sd : ed-sd, **kwargs):
    '''start_date defaults to January 1st of the current year; end_date defaults to the current date;'''
    TODAY = dt.toordinal(dt.today())
    #JAN_FIRST = dt.toordinal(pd.Timestamp(dt.today().year,1,1))
    
    if ytd == False: #Takes dates a face value, does not allow for year-to-date operations
        assert start_col != False and end_col != False, 'If ytd is False, both start- and end-date columns must be provided'
        temp_end_dates = dataframe.loc[:, end_col]
        temp_start_dates = dataframe.loc[:, start_col]
    else: #Replaces end date with TODAY if the date is greater than TODAY, otherwise end date stays the same
        temp_end_dates = np.array([TODAY if dataframe.loc[:, end_col][idx] > TODAY else dataframe.loc[:, end_col][idx] for idx in dataframe.index.tolist()])
        temp_start_dates = dataframe.loc[:, start_col]
    return date_math(temp_end_dates, temp_start_dates) #Executes date math from lambda function: default is end_date minus start_date

#Import data from file
data = pd.read_csv(FULL_PATH, sep='\t', encoding='utf-8') 


###--- Data Engineering Section ---###
data['Plan_Start_Year'] = [int(i[0:4]) for i in data.Plan_Start_Date] #Breakout plans by year
data['Customer_Since_Year'] = [int(i[0:4]) for i in data.Customer_Since] #Breakout customers by year

#Map plan cost to column
plan_cost = []
for plan in data.Plan:
    for key in subscriber_plan.keys():
        if key == plan:
            plan_cost.append(subscriber_plan[key][0]) #Plan cost
        else:
            pass
assert len(plan_cost) == len(data) #Make sure map is correct
data['Plan_Cost'] = plan_cost

data['Plan_Start_Date_Ord'] = date_to_ordinal(data.Plan_Start_Date) #Ordinal column for Plan Start Date

#Find Plan End Date with date duration values based on plan
end_dates = []
for idx in data.index.tolist():
    if data.Plan[idx] == 'Obsidian':
        ed = data.Plan_Start_Date_Ord[idx] + 2*365
    elif data.Plan[idx] == 'Infinity':
        ed = data.Plan_Start_Date_Ord[idx] + 5*365 
    else:
        ed = data.Plan_Start_Date_Ord[idx] + 1*365
    end_dates.append(ed)
data['Plan_End_Date_Ord'] = end_dates #Ordinal column for Plan End Date
data['Plan_End_Date'] = [dt.fromordinal(idx).strftime("%Y-%m-%d") for idx in data.Plan_End_Date_Ord] #Convert end date to string-value date from ordinal
data['Customer_Since_Ord'] = date_to_ordinal(data.Customer_Since) #Ordinal column for Customer Since
data['Plan_End_Year'] = [dt.fromordinal(idx).year for idx in data.Plan_End_Date_Ord] #Strip year from Plan_End_Date
data['Cust_Since_First_Yr'] = [pd.Timestamp(data.Customer_Since_Year[idx],12,31).strftime("%Y-%m-%d") for idx in data.index.tolist()] #December 31st of the Cust_Since_Year
data['Cust_Since_First_Yr_Ord'] = date_to_ordinal(data.Cust_Since_First_Yr) #Ordinal value of Cust_Since_First_Yr
data['Plan_Start_Jan_First'] = [pd.Timestamp(data.Plan_Start_Year[idx],1,1).strftime("%Y-%m-%d") for idx in data.index.tolist()] #January 1st of Plan_Start_Year
data['Plan_Start_Jan_First_Ord'] = date_to_ordinal(data.Plan_Start_Jan_First) #Ordinal of Plan_Start_Jan_First

data['First_Partial_Year'] = date_diff(data, start_col='Customer_Since_Ord', end_col='Cust_Since_First_Yr_Ord', ytd=False, date_math=lambda ed,sd : (ed-sd)//30) #Number of partial months during first year

#Dataframe of plan info (for visualization purposes)
my_planz = pd.DataFrame.from_dict(subscriber_plan, orient='index',columns=['Cost','Duration'])

#DEBUG_Data_Collision() #Run debug for Plan_Start and Cust_Since columns
data.head(1)

In [ ]:
#Map list of all whole years to each customer
TODAY = dt.toordinal(dt.today())

this_year = dt.today().year
last_year = dt.today().year-1

#Put together list of all completed years for each customer
first_comp_yr = []
for idx in data.index.tolist():
    if data.Plan_End_Year[idx] >= this_year:
        first_comp_yr.append(list(range(data.Customer_Since_Year[idx]+1,this_year)))
    elif data.Plan_End_Year[idx] < this_year: #If current plan ended before this current year
        first_comp_yr.append(list(range(data.Customer_Since_Year[idx]+1,data.Plan_End_Year[idx])))
    else:
        first_comp_yr.append(np.nan) #Insert NAN value if nothing else works

#Make blank matrix with all customer years
cust_since_matrix = pd.DataFrame(None, columns=list(range(data.Customer_Since_Year.min(),data.Customer_Since_Year.max()+1)),index=data.index)
csm = cust_since_matrix.copy() #Make a copy to mess around with

#Populate matrix with whole years
for idx, item in enumerate(first_comp_yr):
    if len(item) > 0:
        for subitem in item:
            if subitem in csm.columns.tolist():
                csm.loc[idx,subitem] = 12 #Number of months in every full year
            else:
                pass
    else:
        pass
csm.fillna(0,inplace=True) #Place zeros in all NAN values

#Populate matrix with partial years
for idx in data.index.tolist():
    csm.loc[idx, data.loc[idx,'Customer_Since_Year']] = data.loc[idx, 'First_Partial_Year']
    csm.loc[idx, 2023] = data.loc[idx, 'Last_Partial_Year_Ct']

#Output matrix to file for easy import in next notebook file
csm.to_csv('Months_on_Plan_Historical.csv', sep='\t', encoding='utf-8', index=False)

#Output data to new file with all the new columns
data.to_csv('Customers_Enhanced.csv',sep='\t', encoding='utf-8', index=False)

csm.head()

#Data cleaning/validation for dates -- this is what led to the updated customers.csv file
def DEBUG_Data_Collision():
    data['Data_Collision'] = ['Yes' if data.Plan_Start_Date_Ord[idx] < data.Customer_Since_Ord[idx] else 'No' for idx in data.index.tolist()]
    data_collision = data.query('Data_Collision == "Yes"')
    if len(data_collision) == 0:
        print('No data collision present.')
        pass
    else:
        data_2 = data.query('Data_Collision == "No"')
        data_3 = data.loc[data_2.index, ['Customer_ID','Last_Name','First_Name','Sex','Age', 'ZIP','Plan','Plan_Start_Date','Customer_Since','Salesman_EID']]
        data_3.reset_index(inplace=True, drop=True)
        data_3.to_csv('Customers_Cleaned_2.csv', index=False, encoding='utf-8', sep='\t')
        print('Data collision present in Plan_Start_Date and Cust_Since columns.')
        
    future_tense = data.query(f'Plan_Start_Date_Ord >= {dt.toordinal(dt.today())} and Customer_Since_Year == 2023')
    if len(future_tense) == 0:
        print('No Future plans present.')
        pass
    else:
        new_idx = list(set(data.index) - set(future_tense.index))
        data_3 = data.loc[new_idx, ['Customer_ID','Last_Name','First_Name','Sex','Age', 'ZIP','Plan','Plan_Start_Date','Customer_Since','Salesman_EID']]
        data_3.reset_index(inplace=True, drop=True)
        data_3.to_csv('Customers_Cleaned_2.csv', index=False, encoding='utf-8', sep='\t')

#First attempt at partial year values --> made it way too complicated >:{
output = []
for idx in data.index.tolist():
    if data.Plan_End_Year[idx] >= dt.today().year: #Does plan end after current year?
        last_partial_year = dt.today().year
        temp_end_date = dt.toordinal(dt.today())
        start_date_last_plan_yr = dt.toordinal(pd.Timestamp(dt.today().year,1,1))
    elif data.Plan_End_Year[idx] < dt.today().year: #Does plan end before current year?
        last_partial_year = data.Plan_End_Year[idx]
        start_date_last_plan_yr = dt.toordinal(pd.Timestamp(dt.fromordinal(data.Plan_End_Date_Ord[idx]).year,1,1))
        temp_end_date = data.Plan_End_Date_Ord[idx]

    #output.append([dt.fromordinal(temp_end_date).strftime("%Y-%m-%d"), dt.fromordinal(start_date_last_plan_yr).strftime("%Y-%m-%d")])
    output.append((temp_end_date-start_date_last_plan_yr)//30)

data['Last_Partial_Year_Ct'] = output
data.Last_Partial_Year_Ct[0:5]